<a href="https://colab.research.google.com/github/alimaan2935/Exp/blob/master/tf_od.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%ls

sample_data/


In [3]:
%cd /root/

/root


In [0]:
%ls

# Cloning tf OD repo

In [0]:
!git clone --quiet https://github.com/tensorflow/models.git

# Installing dependencies

In [9]:
%cd ~/models/research

/root/models/research


In [0]:
!apt-get install -qq protobuf-compiler python-tk
!pip install -q Cython contextlib2 pillow lxml matplotlib
!pip install -q pycocotools

In [0]:
!protoc object_detection/protos/*.proto --python_out=.

In [12]:
import os
os.environ['PYTHONPATH']

'/env/python'

In [0]:
os.environ['PYTHONPATH'] += '/root/models/research/:/root/models/research/slim/'

In [14]:
!python setup.py build -q
!python setup.py install -q

running build
running build_py
creating build
creating build/lib
creating build/lib/object_detection
copying object_detection/eval_util.py -> build/lib/object_detection
copying object_detection/export_tflite_ssd_graph_lib.py -> build/lib/object_detection
copying object_detection/inputs_test.py -> build/lib/object_detection
copying object_detection/exporter_test.py -> build/lib/object_detection
copying object_detection/export_tflite_ssd_graph.py -> build/lib/object_detection
copying object_detection/eval_util_test.py -> build/lib/object_detection
copying object_detection/model_main.py -> build/lib/object_detection
copying object_detection/model_hparams.py -> build/lib/object_detection
copying object_detection/model_tpu_main.py -> build/lib/object_detection
copying object_detection/export_tflite_ssd_graph_lib_test.py -> build/lib/object_detection
copying object_detection/model_lib_test.py -> build/lib/object_detection
copying object_detection/export_inference_graph.py -> build/lib/object

## Checking if the api has been installed

In [15]:
!python object_detection/builders/model_builder_test.py

......................
----------------------------------------------------------------------
Ran 22 tests in 0.125s

OK


# Starting model

In [0]:
from zipfile import ZipFile
import os
import requests

import shutil
import glob
import urllib
import tarfile

In [16]:
%cd object_detection
%mkdir ms
%cd ms

/root/models/research/object_detection
/root/models/research/object_detection/ms


In [0]:
%mkdir data
%mkdir model

In [0]:
def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

In [0]:
file_id = '1W-zP8uJDwOZEPqOS17XYA8BxCHEHCCfp'
destination = 'data.zip'
download_file_from_google_drive(file_id, destination)


In [52]:
ds = ZipFile(destination)
ds.extractall()
os.remove(destination)
print('Extracted zip file ' + destination)

Extracted zip file data.zip


In [53]:
%ls data

label_map.pbtxt              val.record-00000-of-00010
train.record-00000-of-00010  val.record-00001-of-00010
train.record-00001-of-00010  val.record-00002-of-00010
train.record-00002-of-00010  val.record-00003-of-00010
train.record-00003-of-00010  val.record-00004-of-00010
train.record-00004-of-00010  val.record-00005-of-00010
train.record-00005-of-00010  val.record-00006-of-00010
train.record-00006-of-00010  val.record-00007-of-00010
train.record-00007-of-00010  val.record-00008-of-00010
train.record-00008-of-00010  val.record-00009-of-00010
train.record-00009-of-00010


In [0]:
MODEL = 'faster_rcnn_inception_v2_coco_2018_01_28'
MODEL_FILE = MODEL + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
DEST_DIR = "model"

In [0]:
if not (os.path.exists(MODEL_FILE)):
  urllib.request.urlretrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)

In [0]:
tar = tarfile.open(MODEL_FILE)
tar.extractall()
tar.close()

In [0]:
os.rename(MODEL, DEST_DIR)

In [0]:
%rm model/pipeline.config

In [59]:
%ls model

checkpoint                 model.ckpt.data-00000-of-00001  model.ckpt.meta
frozen_inference_graph.pb  model.ckpt.index                saved_model/


In [0]:
pipeline = "1dy-_8CB3EWWZBnExiKab1aOzMkMgstKG"
fileName = 'pipeline.config'
download_file_from_google_drive(pipeline, fileName)
%mv pipeline.config model/

In [61]:
%ls model

checkpoint                      model.ckpt.index  saved_model/
frozen_inference_graph.pb       model.ckpt.meta
model.ckpt.data-00000-of-00001  pipeline.config


In [62]:
%cd ..

/root/models/research/object_detection


In [63]:
!python model_main.py \
    --pipeline_config_path=/root/models/research/object_detection/ms/model/pipeline.config \
    --model_dir=/root/models/research/object_detection/ms/model \
    --alsologtostderr \
    --num_eval_steps=500

/usr/local/lib/python3.6/dist-packages/object_detection-0.1-py3.6.egg/object_detection/utils/visualization_utils.py:26: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "model_main.py", line 26, in <module>
    from object_detection import model_lib
  File "/usr/local/lib/python3.6/dist-packages/object_detection-0.1-py3.6.egg/object_detection/model_lib.py", line 27, in <module>
    from object_detection import eval_util
  File "/usr/local/lib/python3.6/dist-packages/object_detection-0.1-py3.6.egg/object_detection/eval_util.py", line 27, in <module>
    from object_detection.metrics import coco_evaluation
  File "/usr/local/lib/python3.6/dist-packages/object_detection-0.1-py3.6.egg/object_detection/metrics/